In [59]:
#Config
from pyspark import SparkConf, SparkContext, HiveContext
import re
import numpy as np
import pandas as pd
import datetime

In [60]:
sc.stop()
conf = SparkConf().set("spark.executor.instances", 32).set("spark.driver.maxResultSize", "8g")
sc = SparkContext(conf=conf)
hc = HiveContext(sc)

In [61]:
# Cyrrilic -> Latin
symbols = (u"абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ",
           u"abvgdeejzijklmnoprstufhzcss_y_euaABVGDEEJZIJKLMNOPRSTUFHZCSS_Y_EUA")

tr = {ord(a):ord(b) for a, b in zip(*symbols)}

In [62]:
#Read data
dfr = pd.read_csv('./data/result_Card_21.csv',sep=';')
df1 = pd.get_dummies(dfr[[u'open', u'click', u'abuse', u'unsubscribe', u'error']])
df1.columns = [c.decode('utf-8').translate(tr).replace(' ','_') for c in df1.columns]
df1['email'] = dfr['email']
df1 = df1.rename(columns = {       u'abuse':u'abuse',
                               u'open_open': u'open',
                             u'click_click': u'click',
                 u'unsubscribe_unsubscribe': u'unsubscribe',
                    u'error_Jaloba_na_spam': u'err_spam',
       u'error_Poctovyj_asik_ne_susestvuet': u'err_not_exists',
           u'error_Poctovyj_asik_neaktiven': u'err_not_active',
                                   u'email':u'email'})
df1[[  u'open',u'click',  u'unsubscribe',   u'err_spam', u'err_not_exists',u'err_not_active']] = \
df1[[  u'open',u'click',  u'unsubscribe',   u'err_spam', u'err_not_exists',u'err_not_active']].astype(int)

1087

In [63]:
#hc.createDataFrame(df1).registerTempTable("deliv_res_temp")
'''
hc.sql('drop table if exists deliv_res_temp1')
hc.registerDataFrameAsTable(hc.createDataFrame(df1), 'deliv_res_temp1')
hc.sql('drop table if exists user_kposminin.delivery_results2')
hc.sql('create table user_kposminin.delivery_results2 as select * from deliv_res_temp')
'''

"\nhc.sql('drop table if exists deliv_res_temp1')\nhc.registerDataFrameAsTable(hc.createDataFrame(df1), 'deliv_res_temp1')\nhc.sql('drop table if exists user_kposminin.delivery_results2')\nhc.sql('create table user_kposminin.delivery_results2 as select * from deliv_res_temp')\n"

In [64]:
process_query = '''
drop table if exists user_kposminin.delivery_results3;

create table user_kposminin.delivery_results3 as 
select 
   d.*,
   not (t.email is NULL) as control,
   s.model,
   s.score,
   not (pa.utm_term is NULL) as started,
   eu.utm_term,
   (pa.is_processed = 3) as completed

from 
   user_kposminin.delivery_results2 d
left join (select `_1` as email from user_mvsurovikov.la_cc_emails_hashed_ctrl_20160809) t
    on d.email = t.email
left join
     (select 
        email,
        model,
        max(score) as score
      from
        user_mvsurovikov.la_cc_scored_20160809_3
      group by 
        email,
        model
      ) s
     on
      d.email = s.email
left join
    (select 
        `_1` as email,
        `_2` as utm_term
    from
        (
        select * from user_mvsurovikov.la_cc_emails_hashed_ctrl_20160809 
        union all
        select * from user_mvsurovikov.la_cc_emails_hashed_20160809 
        ) a
    ) eu
    on d.email = eu.email
left join prod_dds.portal_application pa 
    on eu.utm_term = pa.utm_term'''

#for q in process_query.split(';'):
#    hc.sql(q)

df = pd.DataFrame([r.asDict() for r in hc.sql('select * from user_kposminin.delivery_results3').collect()])


In [65]:
df1 = df
for c in df.columns:
    if df[c].dtype == type(True): df[c] = df[c].astype(int)

In [ ]:
df
#df[[ u'open', u'click', u'unsubscribe', u'err_spam', u'err_not_exists', u'err_not_active', u'control', u'started']].drop_duplicates()

In [ ]:
#df.mean()

In [ ]:
df.groupby(['model','control'])['email'].count()
#df.ix[df[(df['control'] == 1) &(df['model'] == 'NA')].index,'model'] = 'control'

In [ ]:
 %matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
model = 'mkb_cc'
dfm = df[df['model'] == model]

In [ ]:
dfm['scb'] = pd.cut(dfm['score'], bins = dfm['score'].quantile(np.arange(0,1.2,0.2)).values, labels  = False).values


In [ ]:
dfm.groupby('scb').mean()